In [150]:
import requests
import numpy as np
import pandas as pd
import bs4
import re
import json

In [151]:
def parse_link(link):
    
    parsed = bs4.BeautifulSoup(requests.get(link).content, 'html.parser')
    
    return parsed

In [152]:
# make a list of strings unique by appending "_1", "_2", etc. to duplicate entries

def uniquify(df_columns):
    seen = set()

    for item in df_columns:
        fudge = 1
        newitem = item

        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)

        yield newitem
        seen.add(newitem)

In [153]:
# download terms and conditions from a hyperlink

def get_TC(link):
    
    tc_page = parse_link(link)
    
    # Get text from the lines, remove whitespaces, and the first and last lines (title and footnote)
    tc = [re.sub('\s', '', x.get_text()) for x in tc_page.select('td + .bdr-dtail td')][1:-1]
    
    tc_df = pd.DataFrame([tc[1::2]], columns = tc[0:-1:2])
    
    # Change column names from Chinese to English
    tc_df = translate_column_headers(tc_df)
    
    # Make columns unique by appending numbers if there are multiple follow-ons
    tc_df.columns = list(uniquify(tc_df.columns))
    
    return tc_df


In [154]:
# Append two dataframes preserving the column orders of the wider dataframe (direct appendation would change the column order)

def TC_append(df1, df2):
    
    cols = df1.columns if len(df1.columns) > len(df2.columns) else df2.columns
    df = pd.concat([df1, df2])[cols]
    
    return df


In [155]:
def translate_column_headers(df):
    nameDict = {'债券简称': 'Short Name',
                '债券代码': 'Code',
                '发行人': 'Issuer',
                '债券类型': 'Bond Type',
                '发行日期': 'Issue Date',
                '最新评级': 'Latest Rating',
                '债券发行日': 'Issue Date',
                '到期兑付日': 'Maturity Date',
                '上市交易日': 'First Trade Date',
                '债券摘牌日': 'Last Trade Date',
                '债券期限': 'Tenor',
                '流通范围': 'Circulation Range',
                '面值(元)': 'Face Value',
                '发行价格(元)': 'Issue Price',
                '计划发行量(亿)': 'Planned Issue Amount (100 millions)',
                '实际发行量(亿)': 'Actual Issue Amount (100 millions)',
                '币种': 'Currency',
                '计息基础': 'Day Count Basis',
                '息票类型': 'Coupon Type',
                '债券起息日': 'Dated Date',
                '付息频率': 'Coupon Frequency',
                '票面利率(%)': 'Coupon Rate',
                '发行收益率(%)': 'Yield at Issue',
                '参考收益率(%)': 'Yield for Reference',
                '基准利率': 'Reference Rate',
                '基准利差(BP)': 'Spread to Reference Rate', 
                '信用评级机构一': 'Rating Agency 1',
                '债项/主体评级一': 'Issue/Issuer Rating 1',
                '信用评级机构二': 'Rating Agency 2',
                '债项/主体评级二': 'Issue/Issuer Rating 2',
                '行权类型': 'Option Type',
                '行权日期': 'Option Date',
                '托管机构': 'Clearing House',
                '续发行发行日': 'Follow-on Issue Date',
                '续发行上市交易日': ' Follow-on First Trade Date',
                '计划续发行总额(亿)': 'Planned Follow-on Amount (100 millions)',
                '实际续发行总额(亿)': 'Actual Follow-on Amount (100 millions)',
                '续发行价格(元)': 'Follow-on Issue Price',
                '续发行收益率(%)': 'Follow-on Yield at Issue',
                '备注': 'Notes'}

    df.columns = df.columns.to_series().map(lambda x: nameDict[x] if x in nameDict.keys() else x)
    
    return df

In [165]:
# given search criteria, download T&C of all bonds to a dataframe

def get_bond_list(bondType = '', couponType = '', issueYear = '', startPage = 1, endPage = np.nan):
    
    output = pd.DataFrame()
    page = startPage
    
    #parameters for the link
    params = {'bondName':'', 'bondCode':'', 'bondType': bondType, 'issueEnty':'',
             'couponType':couponType, 'issueYear':issueYear, 'enty':'', 'rtngShrt':''}
    
    while True:
        
        params['pagingPage_il_'] = page
        
        response = requests.get('http://www.chinamoney.com.cn/dqs/rest/cm-u-pt/bondInfoList', params = params)
        
        #convert response to a dict
        response_dict = json.loads(response.text)
        
        #check if the loop has reached the last page
        if page > response_dict['data']['totalPages'] or page > endPage:
            break
            
        if page%10 == 1:
            print("Downloading page {}".format(page))
        
        #bond list from the 'records' entry in the dict
        bond_list = pd.DataFrame(response_dict['records'])
    
        #for each bond in the list, download its T&C and combine them into a DataFrame
        
        for code in bond_list['definedCode']: 
            
            link = 'http://www.chinamoney.com.cn/fe/chinamoney/searchDetailInfo.action?entyCode=' + code
            TC = get_TC(link)
            
            # add the T&C link to the output dataframe in case the T&C is blank
            TC['Link'] = link
            
            output = TC_append(output, TC)
        
        page+=1
        
    return output

In [157]:
# download the master mapping table from cninfo

def get_bond_map():
    
    webpage = requests.get('http://www.cninfo.com.cn/disclosure/bondcross-references.jsp')
    
    # encoding parameter is necessary as some characters used are outside of the specified char set
    soup = bs4.BeautifulSoup(webpage.content, 'html.parser', from_encoding='GB18030')
    
    data = [x.get_text() for x in soup.find_all('td', {'class': ['zx_center_title', 'zx_data6']})]
    data = np.array(data).reshape(-1,8)
    
    # put the data into a dataframe, using the first row as column headers
    bond_map = pd.DataFrame(data[1:,:], columns = data[0,:])
    
    return bond_map

In [158]:
# Change the layout of the map to enable lookup

def gather_map(bond_map):
    
    bond_map = pd.melt(bond_map, id_vars = ['债券名称', '到期日期'],
                               value_vars = ['深交所代码', '上交所代码', '银行间代码'],
                               var_name = '交易所', value_name = '代码')
    bond_map['交易所'] = bond_map['交易所'].str.replace('代码', '')
    
    return bond_map

In [159]:
# Map the exchange for each row of a dataframe

def map_exchange(row, bondmap):
    
    if (row['Code'], row['Maturity Date']) in bondmap.index:
        exchange = bondmap.loc[(row['Code'], row['Maturity Date']),'交易所']
    else:
        exchange = np.nan
    
    return exchange

## Functions to download T&C given a list of symbols

In [160]:
# Convert symbols to codes used in the URLs

def find_code(symbol):
    
    parsed = parse_link('http://www.chinamoney.com.cn/fe/chinamoney/seniorSearchAjaxAction.action?bondCode=' 
                         + str(symbol) + '&bondName=')
    code = re.search('[0-9]+',str(parsed))
    
    if code != None:
        code = code.group(0)
    
    return code

In [161]:
def TC_from_symbol(symbol):
    
    code = find_code(symbol)
    link = 'http://www.chinamoney.com.cn/fe/chinamoney/searchDetailInfo.action?entyCode=' + code
    TC = get_TC(link)
        
    return TC

## Function to output

In [163]:
# remove unnecessary columns and write results to csv

def output_list(df, path):
    
    cols = ['Last Trade Date', 'Tenor', 'Circulation Range', 'Planned Issue Amount (100 millions)',
            'Yield for Reference', 'Reference Rate', 'Spread to Reference Rate', 'Rating Agency 1',
            'Issue/Issuer Rating 1', 'Rating Agency 2', 'Issue/Issuer Rating 2']
    
    df.drop(cols, axis = 1).to_csv(path, index = False, encoding = 'utf-8')

In [166]:
interbank_1 = get_bond_list(bondType = 100041, couponType = '', issueYear = 2017, starPage = 1, endPage = 100)

In [169]:
sst.to_csv('C:/Users/oou/Desktop/Analytics/China Bond/Downloads/Jan 2017/sst1.csv', index = False, encoding = 'utf-8', quoting = )